#　小テスト：あるストリーミングサービスの利用改善分析
#  Short Test: Analysis to improve a streaming service

### 下記の作業フローを参考して、データ分析プロジェクトを完成し、次回の面接の際に分析Processと結論を説明してください(20~30分目安)

### Follow the operation below as a reference to complete the data analysis project. In the next interview present the analysis process and your conclusion within 20 to 30 min

* ChatGPTの利用は問題ありません。
* もし利用した場合は、どのタスクで利用したか教えてください。
* 従来のデータ分析のワークフローがどのように変化したか教えてください。

* Using ChatGPT is ok.
* If you use it explain in what case did you use it.
* Considering the analysis workflow up to now what changes did you make.



## リクエスト
* 既に把握している「前提」を使って、必要な「分析結論」を導き出す

## 既に把握した「前提」
* Goal
* 現状

## 欲しい「分析結論」
* 問いの明確化
* 課題の分解
* 課題の設定

## Request
* Starting with what you already know, draw the conclusion of the analysis

## What is already known beforehand
* Goal
* Present condition

## What is desired
* Clarity of the question
* Breaking up the subject into smaller components
* Setting up the subject

## 既に把握している「前提」
* あるストリーミングサービスが2024年1月にリリースされました。
* そのサービスの新規利用者の利用状況を改善する必要があります

## 現状
* 現在、新規利用者の __Monthly Active__ 日数が芳しくありません。
* 新規利用者の60%は利用初日(`log_day_sequence = 1`)しかサービスを利用していません。
* したがって、新規利用者の2日目以降の利用率(`log_day_sequence >= 2`)は40%前後です。

## Goal設定
* 新規利用者の2日目以降の利用率(`log_day_sequence >= 2`)を改善する。

## What is already understood? Initial conditions
* There is a streaming service that started operating in January 2024.
* This service wants to improve the use conditions

## Present state
* Presently, the users number of days __Monthly Active__ is not favorable
* For the new users 60% use the service a single day `log_day_sequence = 1`
* Therefore, the number of new users using the service for more than 2 days `log_day_sequence >= 2` is about 40%.

## Conditions for the Goal
* Improving (increasing) the number of days for new users

## 欲しい「分析結論」

提供した`Data_sample`から上記の状況も把握できるようになっています。次回の面接時に以下の分析結論を説明していただくことを期待しています。

* 問いの明確化
  * 2日目以降の利用率」を改善するために、現在新規利用者が2日目に利用するユーザーとしないユーザーの違いは何ですか？

* 課題の分解
  * その違いから、改善の切り口をどのように分解することが考えられますか？

* 課題設定
  * 分析担当として、もし改善の切り口が複数ある場合、優先順位はどのように考えますか？その考え方を教えてください。
 

## 分析に使うData Sample

* `Data_sample.head()`

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/data_sample.csv')
df.rename(columns={'Unnamed: 0': 'index'}, inplace = True)
df.head()

,index,episode_type,episode_id,show_id,user_id,log_date,view_start_time,view_end_time,view_percent,log_day_sequence
0,0,a,ep00000002,s00000002,uu00000002,2024-02-01,2024-02-01 12:13:10,2024-02-01 12:13:24,0.000,1
1,1,a,ep00000002,s00000002,uu00000016,2024-02-02,2024-02-02 23:32:55,2024-02-02 23:34:01,9.091,1
2,2,a,ep00000002,s00000002,uu00000049,2024-02-03,2024-02-03 12:37:43,2024-02-03 12:38:46,9.091,1
3,3,a,ep00000002,s00000002,uu00000050,2024-02-03,2024-02-03 20:59:06,2024-02-03 21:00:27,27.273,1
4,4,a,ep00000002,s00000002,uu00000050,2024-02-03,2024-02-03 20:56:40,2024-02-03 20:57:09,9.091,1


### Column 説明

| #   | Column            |  Count   |  Non-null |  Dtype |
| ----| ----------------- | -------- | --------- | ------ |
| 0   |  `show_type` showの種類    |  23242   |  non-null | object |
| 1   |  `episode_id` 各エピソードのID  |  23242   |  non-null | object |
| 2   |  `show_id` 各作品のID  |  23242   |  non-null | object |
| 3   |  `user_id` userのunique 情報|  23242   |  non-null | object |
| 4   |  `log_date` 視聴の日付け |  23242   |  non-null | dbdate |
| 5   |  `view_start_time` 視聴開始のTimeStamp　|  23242   |  non-null | datetime64[us] |
| 6   |  `view_end_time` 視聴終了のTimeStamp      |  23242   |  non-null | datetime64[us] |
| 7   |  `view_percent` 今回の視聴、episodeどこまで視聴したというpercentage(最後まで見たら100、半分見たら50)|  23242   |  non-null | float64 |
| 8   |  `log_day_sequence` 視聴の日付けはこのUserのサービス新規利用何日目(1:初日、2:2日目) |  23242   |  non-null | Int64 |


このData SampleはUserの一回の視聴は1Recordとして記録します

例）
`user_id = uu00000004`の`user`、`show_id = s00000004`の作品、(`episode_id = ep00000006, show_type = b` (maybe `episode_type`))のepisode、(`2024-02-01 12:13:10~2024-02-01 12:41:20`)で視聴したら、一行のRecordとして記録します

なので、`Data_sample`の __primary key__　は(`user_id&show_id&episode_id&view_start_time`)